In [14]:
import cv2

In [15]:
# # Load image
# img = cv2.imread("C:/Users/migue/OneDrive/Ambiente de Trabalho/SEMESTER 4 - AI/Group Project/IMAGES TO TEST SHARPNESS/IMAGE3.jpg")

# # Convert to grayscale
# gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# # Increase contrast using Histogram Equalization
# equalized = cv2.equalizeHist(gray)

# # Apply a sharpening kernel
# kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
# sharp = cv2.filter2D(equalized, -1, kernel)

# # Save the result
# cv2.imwrite("C:/Users/migue/OneDrive/Ambiente de Trabalho/SEMESTER 4 - AI/Group Project/IMAGES TO TEST SHARPNESS/IMAGE3_ENHANCED.jpg", sharp)

# ✅Good Model to Upscale

Model name is Upscayl

In [16]:
# import cv2

# img = cv2.imread("C:/Users/migue/OneDrive/Ambiente de Trabalho/SEMESTER 4 - AI/Group Project/IMAGES TO TEST SHARPNESS/IMAGE3_upscayl_5x_upscayl-lite-4x.png")
# gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# equalized = cv2.equalizeHist(gray)
# cv2.imwrite("C:/Users/migue/OneDrive/Ambiente de Trabalho/SEMESTER 4 - AI/Group Project/IMAGES TO TEST SHARPNESS/IMAGE3_upscayl_upscaled.png", equalized)



## Testing

In [17]:
# import cv2
# import os

# # Set your folder with original barcode images
# input_folder = "/Barcodes"        # Change this to your folder name
# output_folder = "/Preprocessed"   # Folder where processed images will be saved

# # Create output folder if it doesn't exist
# os.makedirs(output_folder, exist_ok=True)

# # Supported image types
# valid_extensions = [".jpg", ".jpeg", ".png", ".bmp"]

# # Loop through all files
# for filename in os.listdir(input_folder):
#     if any(filename.lower().endswith(ext) for ext in valid_extensions):
#         img_path = os.path.join(input_folder, filename)
#         img = cv2.imread(img_path)
        
#         if img is None:
#             print(f"❌ Could not read {filename}")
#             continue
        
#         gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#         equalized = cv2.equalizeHist(gray)
        
#         # Save processed image
#         output_path = os.path.join(output_folder, os.path.splitext(filename)[0] + "_upscaled.jpg")
#         cv2.imwrite(output_path, equalized)
#         print(f"✅ Processed: {filename} → {output_path}")


# Barcode detection

In [18]:
# import cv2
# import requests
# import os
# from PIL import Image
# import imagehash

# # --- Roboflow API Setup ---
# API_KEY = "riK5t7eDoQp67Y2kXQGW"
# MODEL_ID = "barcode-detection-brbvg"
# VERSION = "2"
# API_URL = f"https://detect.roboflow.com/{MODEL_ID}/{VERSION}?api_key={API_KEY}"

# # --- Video Input ---
# video_path = "videos/short.mp4"
# cap = cv2.VideoCapture(video_path)
# frame_count = 0
# frame_skip = 10  # Process every 10th frame for performance

# # --- Check video opened ---
# if not cap.isOpened():
#     print("[ERROR] Cannot open video.")
#     exit()

# # Create folder for cropped images
# crop_dir = "cropped"
# os.makedirs(crop_dir, exist_ok=True)

# # Set to keep track of saved image hashes
# saved_hashes = []

# # --- Output Video Setup ---
# output_path = "annotated_output.mp4"
# fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec
# fps = cap.get(cv2.CAP_PROP_FPS)
# frame_size = (640, 480)
# out = cv2.VideoWriter(output_path, fourcc, fps, frame_size)

# while cap.isOpened():
#     ret, frame = cap.read()
#     if not ret:
#         print("[ERROR] Could not read next frame. Exiting...")
#         break

#     # Skip frames
#     if frame_count % frame_skip != 0:
#         frame_count += 1
#         continue

#     # Resize for API
#     resized = cv2.resize(frame, frame_size)

#     # Save resized frame temporarily
#     temp_image_path = "temp.jpg"
#     cv2.imwrite(temp_image_path, resized)

#     # Send to Roboflow API
#     try:
#         with open(temp_image_path, "rb") as img_file:
#             response = requests.post(API_URL, files={"file": img_file})
#         data = response.json()
#         predictions = data.get("predictions", [])
#         print(f"[OK] Frame {frame_count}: {len(predictions)} barcode(s) found.")
#     except Exception as e:
#         print("[ERROR] API call failed:", e)
#         break

#     # Delete temp file
#     if os.path.exists(temp_image_path):
#         os.remove(temp_image_path)

#     # Process predictions
#     for i, pred in enumerate(predictions):
#         x = int(pred["x"])
#         y = int(pred["y"])
#         w = int(pred["width"])
#         h = int(pred["height"])
#         label = pred["class"]

#         # Coordinates
#         x1 = max(0, x - w // 2)
#         y1 = max(0, y - h // 2)
#         x2 = min(resized.shape[1], x + w // 2)
#         y2 = min(resized.shape[0], y + h // 2)

#         # Crop region
#         cropped = resized[y1:y2, x1:x2]
#         if cropped.size == 0:
#             continue  # Skip invalid crops

#         # Convert to PIL and compute hash
#         pil_img = Image.fromarray(cropped)
#         crop_hash = imagehash.average_hash(pil_img)

#         # Fuzzy deduplication: allow some small differences
#         is_unique = all(abs(crop_hash - existing_hash) > 5 for existing_hash in saved_hashes)
#         if is_unique:
#             saved_hashes.append(crop_hash)
#             cropped_filename = os.path.join(crop_dir, f"barcode_{frame_count}_{i}.jpg")
#             pil_img.save(cropped_filename)
#             print(f"[CROP] Saved: {cropped_filename}")
#         else:
#             print("[SKIP] Similar barcode already saved.")

#         # Draw bounding box
#         cv2.rectangle(resized, (x1, y1), (x2, y2), (0, 255, 0), 2)
#         cv2.putText(resized, label, (x1, y1 - 10),
#                     cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

#     # Show and save annotated frame
#     cv2.imshow("Barcode Detection", resized)
#     out.write(resized)

#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         print("[STOP] Stopped by user.")
#         break

#     frame_count += 1

# # --- Cleanup ---
# cap.release()
# out.release()
# cv2.destroyAllWindows()
# print("[DONE] Finished video.")
# print(f"[SAVE] Annotated video saved to: {output_path}")


Now we import the needed libraries:

In [19]:
import os
import cv2
import requests
import numpy as np
from PIL import Image
from pyzbar.pyzbar import decode
import imagehash
import realesrgan
import torch


### Configuration

In [20]:
# ========================
# CONFIGURATION
# ========================
ROBOFLOW_API_KEY = "riK5t7eDoQp67Y2kXQGW"
MODEL_ID = "barcode-detection-brbvg"
VERSION = "2"
API_URL = f"https://detect.roboflow.com/{MODEL_ID}/{VERSION}?api_key={ROBOFLOW_API_KEY}"

UPSCALE_SCRIPT = "path/to/upscale_tool.py"  # e.g. call subprocess if needed


### Functions

In [21]:
# ========================
# STEP 1 — DETECT BARCODE REGION
# ========================
def detect_barcodes(frame):
    ##resized = cv2.resize(frame, (640, 480))
    temp_path = "temp.jpg"
    cv2.imwrite(temp_path, frame)

    with open(temp_path, "rb") as img_file:
        response = requests.post(API_URL, files={"file": img_file})

    os.remove(temp_path)
    data = response.json()
    return data.get("predictions", []), frame

# ========================
# STEP 2 — CROP BARCODE REGION
# ========================
def crop_prediction(image, pred):
    x, y, w, h = map(int, [pred["x"], pred["y"], pred["width"], pred["height"]])
    x1, y1 = max(0, x - w // 2), max(0, y - h // 2)
    x2, y2 = min(image.shape[1], x + w // 2), min(image.shape[0], y + h // 2)
    return image[y1:y2, x1:x2]

# ========================
# STEP 3 — ENHANCE IMAGE LOCALLY
# ========================
def enhance_image(image):
    # Convert PIL Image to NumPy array if necessary
    if not isinstance(image, np.ndarray):
        image = np.array(image)

    # Convert RGB (from PIL) to BGR (for OpenCV)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    denoised = cv2.fastNlMeansDenoising(gray, h=30)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    equalized = clahe.apply(denoised)
    sharpened = cv2.filter2D(equalized, -1, np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]]))
    return sharpened

# ========================
# STEP 4 — UPSCALE IMAGE (LOCAL TOOL)
# ========================
def upscale_image(image):
    # Convert NumPy array to PIL Image if necessary
    if isinstance(image, np.ndarray):
        image = Image.fromarray(image)

    # Load Real-ESRGAN model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = RealESRGAN(device, scale=4)
    model.load_weights('weights/RealESRGAN_x4.pth')  # or use default model loading

    # Enhance the image
    upscaled = model.predict(image)

    return upscaled

# ========================
# STEP 5 — TRY TO READ BARCODE
# ========================
def try_decode_barcode(image):
    enhanced = enhance_image(image)
    barcodes = decode(enhanced)
    return barcodes

### Main loop

In [22]:
def process_frame(frame, frame_num, output_folder="output"):
    predictions, resized = detect_barcodes(frame)
    
    # Create per-frame subfolder
    frame_folder = os.path.join(output_folder, f"frame_{frame_num}")
    os.makedirs(frame_folder, exist_ok=True)

    for i, pred in enumerate(predictions):
        cropped = crop_prediction(resized, pred)
        pil_img = Image.fromarray(cropped)
        hash_val = imagehash.average_hash(pil_img)

        # Save with barcode index inside the frame folder
        base_filename = f"barcode_{i}.jpg"
        crop_path = os.path.join(frame_folder, base_filename)
        pil_img.save(crop_path)

        # Try upscaling in loop until barcode is found
        attempt = 0
        success = False
        image = pil_img
        while attempt < 20:
            if attempt > 0 :
                image = enhance_image(image)
            barcodes = decode(image)
            if barcodes:
                for barcode in barcodes:
                    print(f"✅ Success [{attempt}] — {barcode.data.decode('utf-8')}")
                success = True
                break
            attempt += 1

        if not success:
            print(f"❌ Failed to read barcode for {base_filename} in frame {frame_num}")


### EXAMPLE USAGE

In [23]:
# ========================
# EXAMPLE USAGE
# ========================
if __name__ == "__main__":
    video_path = "videos/short.mp4"
    cap = cv2.VideoCapture(video_path)
    frame_num = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if frame_num % 10 == 0:  # every 10th frame
            print(f"\n🎞️ Processing frame {frame_num}")
            process_frame(frame, frame_num)
        frame_num += 1

    cap.release()


    



🎞️ Processing frame 0

🎞️ Processing frame 10

🎞️ Processing frame 20

🎞️ Processing frame 30

🎞️ Processing frame 40

🎞️ Processing frame 50

🎞️ Processing frame 60

🎞️ Processing frame 70

🎞️ Processing frame 80

🎞️ Processing frame 90

🎞️ Processing frame 100

🎞️ Processing frame 110

🎞️ Processing frame 120

🎞️ Processing frame 130

🎞️ Processing frame 140

🎞️ Processing frame 150

🎞️ Processing frame 160

🎞️ Processing frame 170

🎞️ Processing frame 180

🎞️ Processing frame 190

🎞️ Processing frame 200

🎞️ Processing frame 210
❌ Failed to read barcode for barcode_0.jpg in frame 210

🎞️ Processing frame 220
❌ Failed to read barcode for barcode_0.jpg in frame 220

🎞️ Processing frame 230
❌ Failed to read barcode for barcode_0.jpg in frame 230

🎞️ Processing frame 240
❌ Failed to read barcode for barcode_0.jpg in frame 240

🎞️ Processing frame 250
❌ Failed to read barcode for barcode_0.jpg in frame 250

🎞️ Processing frame 260
❌ Failed to read barcode for barcode_0.jpg in frame 260


KeyboardInterrupt: 